In [1]:
try:
    from ase.db import connect
except ImportError:
    !pip install ase
    from ase.db import connect

try:
    import plotly 
except ImportError:
    !pip install plotly
import plotly.graph_objects as go
from utils import *
import pandas as pd
import numpy as np

In [14]:
all_data_dict={}
database_counter=0
for orient in ['110', '100', '111']:
    for term in termination_types[orient]:
        this_db = connect('2dpv_set_'+orient+'_'+term+'.db')
        for i in range(len(A_site_list)): 
            for count_a, a in enumerate(A_site_list[i]):
                for b in B_site_list[i]:
                    for c in C_site_list[i]:
                        for thick in [3,5,7,9]:
                            system_name = a + b + c
                            uid = system_name + '3_' + str(orient) + "_" + str(term) + "_" + str(thick)
                            row = None
                            row = this_db.get(selection=[('uid', '=', uid)])
                            
                            is_metal = None
                            direct_band_gap_energy = None 
                            indirect_band_gap_energy = None
                            electron_effective_mass = None
                            hole_effective_mass = None
                            
                            if row is not None:
                                try:
                                    if not row.data['band_structure']['is_metal']:
                                        if row.data['band_structure']['direct_band_gap']:
                                            Eg = row.data['band_structure']['direct_band_gap_energy']
                                            color = '#00743F'
                                        else:
                                            Eg = row.data['band_structure']['indirect_band_gap_energy']
                                            color = '#F2A104'
                                            
                                        __this_mass=[]
                                        for item in row.data['band_structure']['electron_eff_mass']:
                                            if str(item['spin'])=='1':
                                                __this_mass.append(item['eff_mass'])
                                        if __this_mass != []:
                                            electron_effective_mass=np.average(__this_mass)
                                        
                                        __this_mass=[]
                                        for item in row.data['band_structure']['hole_eff_mass']:
                                            if str(item['spin'])=='1':
                                                __this_mass.append(item['eff_mass'])
                                        if __this_mass != []:
                                            hole_effective_mass=np.average(__this_mass)
                                        
                                        if (hole_effective_mass is not None) and (electron_effective_mass is not None):
                                            database_counter += 1
                                            #print(row.data['band_structure']['direct_band_gap'],color)

                                            this_data_dict = {
                                                'system':uid,
                                                'chemical_class':i+1,
                                                'm_e':abs(electron_effective_mass),
                                                'm_h':abs(hole_effective_mass),
                                                'Eg':Eg,
                                                'color':color
                                            }
                                            all_data_dict[database_counter]=this_data_dict
                                    else:
                                        is_metal = True
                                except KeyError:
                                    pass
df = pd.DataFrame(all_data_dict).T

In [15]:
chemical_classes = df['chemical_class'].unique().tolist()
df_categorized = {}
for cc in chemical_classes:
    this_key = str(cc)
    this_set = df[(df['chemical_class']==cc)]
    if not this_set.empty:
        df_categorized[this_key] = this_set

## Electron Effective Masses against Bandgap Energies 

In [25]:
import copy
fig = go.Figure()

sorted_keys = list(sorted(df_categorized.keys()))
display_this=[False for _ in sorted_keys]
button_list=[]

for i,k in enumerate(sorted_keys):
    fig.add_trace(go.Scatter(x=df_categorized[k]['Eg'],
                             y=df_categorized[k]['m_e'],
                             #visible=True,
                             opacity=0.7,
                             marker=dict(size=12,color=df_categorized[k]['color']),
                             marker_symbol='circle',#name=col,
                             text=df_categorized[k]['system'],
                             name=k
                            )
                 )
    _display_this = copy.deepcopy(display_this)
    _display_this[i] = True
    button_list.append(dict(label = k,
                            method = 'update',
                              args = [{'visible': _display_this},
                                      {'showlegend':False}]))



fig.update_layout(width=900, height=800,
    updatemenus=[go.layout.Updatemenu(
        active=0,
        buttons=button_list)])
    
    
fig.update_xaxes(title_text='$E_{g}$')
fig.update_yaxes(title_text='$|m_e^*|$',type='log')
fig.update_traces(mode='markers', marker_line_width=2, marker_size=10)
fig.show()

## Hole Effective Masses against Bandgap Energies 

In [26]:
import copy
fig = go.Figure()

sorted_keys = list(sorted(df_categorized.keys()))
display_this=[False for _ in sorted_keys]
button_list=[]

for i,k in enumerate(sorted_keys):
    fig.add_trace(go.Scatter(x=df_categorized[k]['Eg'],
                             y=df_categorized[k]['m_h'],
                             #visible=True,
                             opacity=0.7,
                             marker=dict(size=12,color=df_categorized[k]['color']),
                             marker_symbol='circle',#name=col,
                             text=df_categorized[k]['system'],
                             name=k
                            )
                 )
    _display_this = copy.deepcopy(display_this)
    _display_this[i] = True
    button_list.append(dict(label = k,
                            method = 'update',
                              args = [{'visible': _display_this},
                                      {'showlegend':False}]))



fig.update_layout(width=900, height=800,
    updatemenus=[go.layout.Updatemenu(
        active=0,
        buttons=button_list)])
    
    
fig.update_xaxes(title_text='$E_{g}$')
fig.update_yaxes(title_text='$|m_h^*|$',type='log')
fig.update_traces(mode='markers', marker_line_width=2, marker_size=10)
fig.show()